In [1]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from pathlib import Path
import torch.nn.functional as F
import sys
from sklearn.metrics import accuracy_score, recall_score
import random
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from collections import Counter
import re
import requests

In [2]:
import cv2

In [6]:
url = "http://netdissect.csail.mit.edu/dissect/zoo/resnet-152-torch-places365.caffemodel"
save_path = "/path/to/your/directory/resnet-152-torch-places365.caffemodel"

def download_file(url, save_path):
    if os.path.exists(save_path):
        print("The file already exists, no need to download.")
    else:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print("File download successful, save path is:", save_path)
        else:
            print("File download failed, status code:", response.status_code)

files_to_download = [
    {
        "url": "http://netdissect.csail.mit.edu/dissect/zoo/resnet-152-torch-places365.caffemodel",
        "save_path": "../../models/resnet-152-torch-places365.caffemodel"
    },
    {
        "url": "http://netdissect.csail.mit.edu/dissect/zoo/resnet-152-torch-places365.prototxt",
        "save_path": "../../models/resnet-152-torch-places365.prototxt"
    }
]

for file in files_to_download:
    download_file(file["url"], file["save_path"])

File download successful, save path is: ../../models/resnet-152-torch-places365.caffemodel
File download successful, save path is: ../../models/resnet-152-torch-places365.prototxt


In [4]:
# Define the path for the model architecture and weight files.
file_location_path = Path.cwd()
prototxt_path = '../../models/resnet-152-torch-places365.prototxt'
caffemodel_path = '../../models/resnet-152-torch-places365.caffemodel'
project_base_path = file_location_path.parent.parent
ns6_wiki_paths = project_base_path / 'data' / 'processed' / 'landscape_or_not'
image_folder = Path('/home/ubuntu/landscape-aesthetics')

results = []
# Load Caffe model.
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

category_mapping = {}
with open('../../data/external/Places365/categories_places365.txt', 'r') as file:
    for line in file:
        parts = line.strip().split()
        category_name = parts[0][3:]
        category_id = int(parts[1])
        category_mapping[category_id] = category_name

pattern = re.compile(r"^ns6_clean_(\d+)\.csv$") 

for file_name in tqdm(os.listdir(ns6_wiki_paths), desc="Processing files"):
    file_path = ns6_wiki_paths / file_name
    if file_path.is_file() and pattern.match(file_name):
        label_file_path = Path(file_name)
        labeled_csv_name = label_file_path.with_suffix('.csv')
        result = project_base_path / 'data' / 'processed' / 'landscape_type' / labeled_csv_name
        result.parent.mkdir(parents=True, exist_ok=True)

        results = []

        data = pd.read_csv(file_path, usecols=['image_path', 'prediction'])
        data = data[data['prediction'] == 1]
        image_paths = data['image_path'].tolist()
        
        for img_path in image_paths:
            image_path = image_folder / img_path
            
            try:
                # Read and process the image
                image = cv2.imread(str(image_path))
                if image is None:
                    print(f"Warning: Unable to load image {img_path}, skipping...")
                    continue  # Skip if image is not loaded

                # Resize the image
                image = cv2.resize(image, (224, 224))

                # Create a blob from the image
                blob = cv2.dnn.blobFromImage(
                    image,
                    scalefactor=1.0,
                    size=(224, 224),
                    mean=(123.675, 116.28, 103.53),
                    swapRB=True,
                    crop=False
                )

                net.setInput(blob)
                output = net.forward()

                # Extract top 5 predictions
                top5_indices = output[0].argsort()[-5:][::-1]
                top5_probabilities = output[0][top5_indices]
                top5_categories = [category_mapping[idx] for idx in top5_indices]

                # Collect results for this image
                row = [img_path]
                for category, probability in zip(top5_categories, top5_probabilities):
                    row.extend([category, probability])
                results.append(row)

            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
                continue  # If any error occurs, skip this image and proceed to the next one

        # Save results to CSV
        columns = ["filename", "prediction1", "probability1", "prediction2", "probability2",
                   "prediction3", "probability3", "prediction4", "probability4", 
                   "prediction5", "probability5"]

        df = pd.DataFrame(results, columns=columns)
        df.to_csv(result, index=False)


Processing files:   0%|                                 | 0/101 [00:57<?, ?it/s]


KeyboardInterrupt: 